# Download Data for baseline

In [13]:
import wget
url = r"https://tickettagger.blob.core.windows.net/datasets/dataset-labels-top3-30k-real.txt"
output = r"../dataset/dataset-labels-top3-30k-real.txt"
wget.download(url,output)

url = r"https://gist.githubusercontent.com/rafaelkallis/6aa281b00d73d77fc843bd34f8184854/raw/8c10ebf2fd6f937f8667c660ea33d122bac739eb/issues.txt"
output = r"../dataset/dataset_balanced.txt"
wget.download(url,output)

'../dataset/dataset-labels-top3-30k-real (1).txt'

## look at pivot table of data from other repos

In [23]:
import pandas as pd
# custom data set
data_path = r"..\dataset\multiple_projects_benchmark_data.csv"
data_out_path = r"..\dataset\CLEAN_multiple_projects_benchmark_data.csv"
df = pd.read_csv(data_path)
df = df.dropna()
df.head()


,f0_,repo_name
0,__label__bug Single most recently added Custom...,https://api.github.com/repos/qgis/QGIS
1,__label__bug layer style effects not compatibl...,https://api.github.com/repos/qgis/QGIS
2,__label__bug QGIS 3.10.0 fails to launch on Wi...,https://api.github.com/repos/qgis/QGIS
3,__label__bug TIN interpolation in Model Builde...,https://api.github.com/repos/qgis/QGIS
4,__label__bug Map Tip Position / Follow the Hyp...,https://api.github.com/repos/qgis/QGIS


In [147]:
# import pandas as pd

# data_path = r"..\dataset\dataset_balanced.txt"
# data_out_path = r"..\dataset\CLEAN_balanced_data.csv"

# df = pd.read_csv(data_path,names=["f0_"])
# df = df.dropna()
# df = df.drop(index=0).reset_index()
# df.head()

In [148]:
# import pandas as pd

# data_path = r"..\dataset\dataset-labels-top3-30k-real.txt"
# data_out_path = r"..\dataset\CLEAN_UNbalanced_data.csv"

# df = pd.read_csv(data_path,names=["f0_"])
# df = df.dropna()
# df = df.drop(index=0).reset_index()
# df.head()
# df.drop(columns=['index'])

In [149]:
# df['labels'] = df['f0_'].str.split(" ").map(lambda x: x[0])
# df = df.rename(columns={'f0_':'descriptions'})
# print(df.columns)
# df.head()

In [150]:
# df.groupby(['repo_name','labels']).agg({'descriptions':'count'})
# df.groupby(['labels']).agg({'descriptions':'count'})

In [24]:
# Cleaning the text
df_a = df['f0_'].str.split(r'(__label__enhancement)|(__label__bug)|(__label__question)', expand=True)
# retrieve data to separate dataframe for each labels

df_b[1] = df_a[1]
df_b[2] = df_a[2]
df_b[3] = df_a[3]
df_b[4] = df_a[4]
df_b['repo_name'] = df['repo_name']

df = df_b

enh_df = df[df[1] == '__label__enhancement'][[1, 4,'repo_name']]
bug_df = df[df[2] == '__label__bug'][[2, 4,'repo_name']]
question_df = df[df[3] == '__label__question'][[3, 4,'repo_name']]

print(enh_df.head())
print(bug_df.head())
print(question_df.head())

# assign column names to each df
enh_df.columns = ['labels', 'descriptions','repo_name']
bug_df.columns = ['labels', 'descriptions','repo_name']
question_df.columns = ['labels', 'descriptions','repo_name']

print(enh_df.head())
print(bug_df.head())
print(question_df.head())

# remove the prefix '__label__' string from column labels
enh_df['labels'] = enh_df['labels'].str.replace('__label__', '')
bug_df['labels'] = bug_df['labels'].str.replace('__label__', '')
question_df['labels'] = question_df['labels'].str.replace('__label__', '')

# strip away and leading & trailing spaces
enh_df['descriptions'] = enh_df['descriptions'].str.strip(' ')
bug_df['descriptions'] = bug_df['descriptions'].str.strip(' ')
question_df['descriptions'] = question_df['descriptions'].str.strip(' ')

# concatenate each df to single dataframe
df = pd.concat([enh_df, bug_df, question_df])

df.head()


                         1                                                  4  \
4625  __label__enhancement   clarify SDL_LockTexture() description  # This...   
4655  __label__enhancement   [Patch] Add Stadia controller to gamecontroll...   
4657  __label__enhancement   FEATURE: SDL_OpenURL()  # This bug report was...   
4730  __label__enhancement   SDL Mutex Implementation Subpar  # This bug r...   
4750  __label__enhancement   Please allow desktop OpenGL contexts under EG...   

                                        repo_name  
4625  https://api.github.com/repos/libsdl-org/SDL  
4655  https://api.github.com/repos/libsdl-org/SDL  
4657  https://api.github.com/repos/libsdl-org/SDL  
4730  https://api.github.com/repos/libsdl-org/SDL  
4750  https://api.github.com/repos/libsdl-org/SDL  
              2                                                  4  \
0  __label__bug   Single most recently added Custom GDAL "Creat...   
1  __label__bug   layer style effects not compatible with inv

,labels,descriptions,repo_name
4625,enhancement,clarify SDL_LockTexture() description # This ...,https://api.github.com/repos/libsdl-org/SDL
4655,enhancement,[Patch] Add Stadia controller to gamecontrolle...,https://api.github.com/repos/libsdl-org/SDL
4657,enhancement,FEATURE: SDL_OpenURL() # This bug report was ...,https://api.github.com/repos/libsdl-org/SDL
4730,enhancement,SDL Mutex Implementation Subpar # This bug re...,https://api.github.com/repos/libsdl-org/SDL
4750,enhancement,Please allow desktop OpenGL contexts under EGL...,https://api.github.com/repos/libsdl-org/SDL


In [135]:
import pandas as pd
from sklearn.model_selection import train_test_split
import neattext.functions as nfx
import re

In [136]:
# remove any non-english samples
# reference: https://stackoverflow.com/a/150078/9960491
reg_obj = re.compile(r'[^\u0000-\u007F]+', re.UNICODE)
def is_english(desc):
    return (False if reg_obj.match(desc) else True)

df['is_english'] = df['descriptions'].apply(is_english)
df = df[df['is_english']][['labels', 'descriptions']]

In [137]:
# remove the stopwords, emojis from the text and convert it into lower case
def neatify_text(text):
    text = str(text).lower()
    text = nfx.remove_stopwords(text)
    text = nfx.remove_emojis(text)
#     text = nfx.remove_puncts(text)
    return text

df['descriptions'] = df['descriptions'].apply(neatify_text)

In [138]:
# split dataset to train and dev samples
RANDOM_SEED = 12
x_train, x_test, y_train, y_test = train_test_split(df['descriptions'], df['labels'],
                                                 test_size=0.3,
                                                 random_state=RANDOM_SEED)
df_train = pd.concat([x_train, y_train], axis=1).reset_index(drop=True)
df_eval = pd.concat([x_test, y_test], axis=1).reset_index(drop=True)


In [139]:
df.groupby(['labels']).agg({'descriptions':'count'})
print(df.head())

        labels                                       descriptions
0  enhancement  subpasses msl hlsl sorry issues  given followi...
1  enhancement  collect new discovered nodes collect new added...
2  enhancement                         lock property : properties
4  enhancement                          randomize: remove sliders
8  enhancement  feature request: local invoke inside action co...


In [140]:
# # # save train and eval dataframes to csv
# df_train.to_csv(r"../dataset/Mult_train.csv")
# df_eval.to_csv(r"../dataset/Mult_eval.csv")
# df.to_csv(data_out_path)

In [141]:
# save train and eval dataframes to csv
# df_train.to_csv(r"../dataset/balanced_train.csv")
# df_eval.to_csv(r"../dataset/balanced_eval.csv")
# df.to_csv(data_out_path)

In [142]:
# save train and eval dataframes to csv
df_train.to_csv(r"../dataset/unbalanced_train.csv",index=False)
df_eval.to_csv(r"../dataset/unbalanced_eval.csv",index=False)
df.to_csv(data_out_path,index=False)